In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
from sklearn import datasets

#Importacion de un solo flujo de datos
from sklearn.pipeline import Pipeline

# El SVM es sensible a datos no escalados
from sklearn.preprocessing import StandardScaler

In [9]:
iris = datasets.load_iris()
X = iris['data'][:, (2, 3)]
y = (iris.target == 0) #setosa

In [17]:
from sklearn.base import BaseEstimator

class MyLinearStimator(BaseEstimator):
    def __init__(self, c = 1, eta0 = 1, n_epochs = 1000, eta_d = 10000, random_state = None):
        self.C = c
        self.eta0 = eta0
        self.n_epochs = n_epochs
        self.etad = eta_d
        self.random_state = random_state
    
    def eta(self, epoch):
        return self.eta0 / (epoch + self.etad)
    
    def fit(self, X, y):
        if self.random_state: np.random.seed(self.random_state)
        w = np.random.randn(X.shape[1], 1)
        b = 0
        
        t = np.array(y, dtype = np.float64).reshape(-1,1) * 2 - 1
        
        X_t = X * t
        
        self.Js = [] # Cost function log
        
        for epoch in range(self.n_epochs):
            support_vectors_idx = (X_t.dot(w) + t * b <= 1).ravel()
            X_t_sv = X_t[support_vectors_idx]
            t_sv = t[support_vectors_idx]
            
            J = (1/2) * (w * w).sum() + self.C * ((1 - X_t_sv.dot(w) - b * t_sv).sum())
            
            self.Js.append(J)
            
            w_gradient_vector = w - self.C * X_t_sv.sum(axis = 0).reshape(-1, 1)
            b_derivative = -self.C * t_sv.sum()
            
            w = w - self.eta(epoch) * w_gradient_vector
            b = b - self.eta(epoch) * b_derivative
        
        self.intercept = np.array([b])
        self.coef = np.array([b])
        
        support_vectors_idx = (X_t.dot(w) + t * b <= -1).ravel()
        self.support_vectors = X[support_vectors_idx]
        return self
    
    def decision_function(self, X):
        return X.dot(self.coef_[0] + self.intercept[0])
    
    def predict(self, X):
        return self.decision_function(X) >= 0;
    